In [7]:
import os
import sys
import csv
import datetime
import shutil
import random
import glob
import pandas as pd
from multiprocessing import Pool
import numpy as np
from scipy import stats
import subprocess

In [2]:
# formatted by lzg

In [3]:
#########
rbp_dir = "/home/galaxy/data/RBP/Starbase2/2019_2_20"
lncRNA = "%s/LncRNA_total.txt" % rbp_dir
mRNA = "%s/mRNA_total.txt" % rbp_dir
Pseudogene = "%s/Pseudogene_total.txt" % rbp_dir
SncRNA = "%s/SncRNA_total.txt" % rbp_dir
###
files = [lncRNA, mRNA, Pseudogene, SncRNA]
# rbp = os.path.join(rbp_dir, "")
# os.system("cat %s > %s" % (files, ))
#########
tmp_dir = "/home/galaxy/data/RBP/Starbase2/2019_1_22/tmp"
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
result_dir = "/home/galaxy/data/RBP/Starbase2/2019_2_20/GRCh38_bed_dir"

#### Generate Data Table

In [9]:
###################################################################################
rbp_dir ="/home/galaxy/data/RBP/Starbase2/2019_2_20/GRCh38_merged_dir/"
eQTL_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/eQTL/m6a_qtl_bed/"
result_dir = "/home/galaxy/project/QTL_analysis/related_with_RBP/Starbase2/data_table"
######################################################################################
rbp_list = glob.glob("%s/*.bed" % rbp_dir)
eqtl_list = glob.glob("%s/*.bed" % eQTL_dir)
for eqtl in eqtl_list:
    for rbp in rbp_list:
        basename = "%s-%s" % (os.path.basename(eqtl).split(".bed")[0], os.path.basename(rbp))
        result_file = os.path.join(result_dir, basename)
        os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (eqtl, rbp, result_file))

#### Enrichment analysis

In [4]:
rbp_dir = "/home/galaxy/data/RBP/Starbase2/2019_2_20/GRCh38_merged_dir/"
eQTL_dir = "/home/galaxy/data/QTL/m6a_related_SNPs/eQTL/m6a_qtl_bed/"
HapMap_snp = "/home/galaxy/data/QTL/HapMap_SNP/autosomal_GRCh38_snp.bed"
result_dir = "/home/galaxy/project/QTL_analysis/related_with_RBP/Starbase2/Fisher_test"
rbp_list = glob.glob("%s/*.bed" % rbp_dir)
eQTL_list = glob.glob("%s/*.bed" % eQTL_dir)
HapMap_NUMBER = 81039058

In [8]:
# run this at tonight 20:00
def write_to_file(res_list, res_file):
    res_list = ["%s\n" % x.get() for x in res_list]
    with open(res_file, "w") as fw:
        fw.writelines(res_list)

def fisher_test(eqtl_bed, RPB_bed):
         #              RPB_bed       x
        # QTL_SNP         a       m6a - a
        # HapMap_SNP      c       con - c
    QTL_num = stat_file_line_numbers(eqtl_bed)
    a = os.popen("bedtools intersect -a %s -b %s -wa |sort| uniq | wc -l" % (eqtl_bed, RPB_bed)).read()
    c = os.popen("bedtools intersect -a %s -b %s -wa |sort| uniq | wc -l" % (HapMap_snp, RPB_bed)).read()
    b, d = QTL_num - int(a), HapMap_NUMBER - int(c)
    oddsratio, pvalue = stats.fisher_exact([[int(a), b], [int(c), d]])
    line="%s\t%s\t%f\t%f"%(os.path.basename(eqtl_bed).split("-")[0], os.path.basename(RPB_bed), oddsratio, pvalue)
    return line

def stat_file_line_numbers(in_file):
    raw_return = os.popen("wc -l %s" % in_file)
    line_num = int(raw_return.read().split()[0])
    return line_num

#####
for eqtl in eQTL_list:
    res_list, pool = [], Pool(processes=30)
#     for mirna in mirna_list:
    for rbp in rbp_list:
#         res_list = fisher_test(eqtl, rbp)
        res = pool.apply_async(fisher_test, (eqtl, rbp, ))
        res_list.append(res)
    pool.close()
    pool.join()
    write_to_file(res_list, os.path.join(result_dir, os.path.basename(eqtl).split("-")[0]))